In [ ]:
from genepy.epigenetics import chipseq as chip
from genepy.utils import helper as h
import pandas as pd
import os 
import dalmatian as dm # give link to dalmatian
from depmapomics.config_prod import *
import multiprocessing
%load_ext autoreload
%autoreload 2


In [ ]:
! gsutil cp gs://fc-4a2230c7-3b51-4476-8c82-84c15d3962f8/filtering.bed /tmp/
CORES=16

In [ ]:
wm = dm.WorkspaceManager(WGSWORKSPACE)
samp = wm.get_samples()
vcfs = samp['cnn_filtered_vcf']
vcfslist = vcfs[~vcfs.isna()].tolist()
# load vcfs using dalmatian (column: cnn_filtered_vcf)
h.createFoldersFor('/tmp/vcfs/')
guides_bed = pd.read_csv("/tmp/filtering.bed", sep='\t', header=None, names=['chrom', 'start', 'end', 'foldchange'])

In [ ]:
cmd = [
"gsutil cp "+sam+" /tmp/vcfs/"+sam.split('/')[-1]+"&& \
 bcftools index /tmp/vcfs/"+sam.split('/')[-1]+" && \
 bcftools query \
  --exclude \"FILTER!='PASS'&GT!='mis'&GT!~'\.'\" \
  --regions-file /tmp/filtering.bed \
  --format '%CHROM\\t%POS\\t%END\\t%ALT{0}\n' /tmp/vcfs/"+sam.split('/')[-1]+" >\
 /tmp/vcfs/loc_"+sam.split('/')[-1].split('.')[0]+".bed &&\
 rm /tmp/vcfs/"+sam.split('/')[-1]+"*" for sam in vcfslist]
h.parrun(cmd, cores=CORES)

In [ ]:
def stuff(file):    
    bed = pd.read_csv('/tmp/vcfs/'+file, sep='\t', header=None, names=['chrom', 'start', 'end', 'foldchange'])
    bed['foldchange'] = 1
    name = file.split('/')[-1].split('.')[0].split('_')[1]
    if len(bed)==0:
        return (name, None)
    val = chip.putInBed(guides_bed, bed, mergetype='sum')
    return (name, val)

pool = multiprocessing.Pool(CORES)
res = pool.map(stuff, os.listdir('/tmp/vcfs/'))
#res = [stuff(val) for val in os.listdir('/tmp/vcfs/')[5:]]
for name, val in res:
    if val is not None:
        guides_bed[name] = val
guides_bed.to_csv('binary_mutguides.tsv', sep='\t')